### **02 — Data Integration & Feature Building - Euroleague**
### **Goal**

Create a game-level table with home/away features and a home_win label. Inputs are Stage-1 cleaned files in clean_data/.

1) Setup & load

In [8]:
import pandas as pd
import numpy as np
from pathlib import Path

CLEAN = Path("clean_data")

hdr = pd.read_csv(CLEAN / "clean_header.csv", low_memory=False)
box = pd.read_csv(CLEAN / "clean_boxscore.csv", low_memory=False)
teams = pd.read_csv(CLEAN / "clean_teams.csv", low_memory=False)


### 2) Helper: pick columns by “best available” name

(Minor schema variations? This keeps things robust.)

In [9]:
def pick(df, options):
    for c in options:
        if c in df.columns: 
            return c
    return None

# Likely keys (based on your notes)
GAME_ID = pick(hdr, ["game_id", "game", "gamecode"])
TEAM_A  = pick(hdr, ["team_id_a", "team_a_id", "home_team_id", "hometeam_id"])
TEAM_B  = pick(hdr, ["team_id_b", "team_b_id", "away_team_id", "awayteam_id"])

# Boxscore keys
BOX_GAME = pick(box, ["game_id", "game", "game_i"])
BOX_TEAM = pick(box, ["team_id", "team"])
BOX_PLAYER = pick(box, ["player_id", "player"])

assert GAME_ID and TEAM_A and TEAM_B, "Header key columns not found."
assert BOX_GAME and BOX_TEAM, "Boxscore key columns not found."


### **3) Label & basic game master**

In [10]:
# score columns (names vary a bit by dataset version)
HOME_SCORE = pick(hdr, ["score_a", "home_score", "homescore", "final_score_a"])
AWAY_SCORE = pick(hdr, ["score_b", "away_score", "awayscore", "final_score_b"])
assert HOME_SCORE and AWAY_SCORE, "Score columns not found in header."

game_master = hdr[[GAME_ID, TEAM_A, TEAM_B, HOME_SCORE, AWAY_SCORE, "season_code"]].drop_duplicates()

# Winner label
game_master["home_win"] = (game_master[HOME_SCORE].astype(float) > game_master[AWAY_SCORE].astype(float)).astype(int)


### **4) Optional: enrich with readable team names**

(this just makes the dataset nicer to inspect.)

In [11]:
TEAM_ID = pick(teams, ["team_id", "teamcode", "id"])
TEAM_NAME = pick(teams, ["team_name", "team", "name"])
TEAM_ABBR = pick(teams, ["team_abbreviation", "abbreviation", "abbr"])

teams_slim = teams[[c for c in [TEAM_ID, TEAM_NAME, TEAM_ABBR] if c]].drop_duplicates()

# Map names for A/B
if TEAM_ID:
    game_master = game_master.merge(
        teams_slim.add_prefix("a_"), left_on=TEAM_A, right_on=f"a_{TEAM_ID}", how="left"
    ).merge(
        teams_slim.add_prefix("b_"), left_on=TEAM_B, right_on=f"b_{TEAM_ID}", how="left"
    )

# Nice-to-have human labels:
game_master.rename(columns={
    f"a_{TEAM_NAME}": "home_team_name",
    f"b_{TEAM_NAME}": "away_team_name",
    f"a_{TEAM_ABBR}": "home_abbr",
    f"b_{TEAM_ABBR}": "away_abbr",
}, inplace=True)


### **5) Aggregate boxscore to team-game totals**

We’ll sum player rows → team totals, then derive percentages.

In [12]:
# numeric columns we care about (only keep if present)
num_candidates = [
    "points","assists","steals","turnovers","blocks_favour","blocks_against",
    "fouls_committed","fouls_received",
    "offensive_rebounds","defensive_rebounds",
    "two_points_made","two_points_attempted",
    "three_points_made","three_points_attempted",
    "free_throws_made","free_throws_attempted",
    # some datasets have alt names:
    "rebounds","total_rebounds","valuation"
]
num_cols = [c for c in num_candidates if c in box.columns]

box_team = (box
    .groupby([BOX_GAME, BOX_TEAM, "season_code"], dropna=False)[num_cols]
    .sum(min_count=1)
    .reset_index()
)

# derive safe percentages
def safe_div(a, b):
    return a / b.replace(0, np.nan)

if {"two_points_made","two_points_attempted"}.issubset(box_team.columns):
    box_team["fg2_pct"] = safe_div(box_team["two_points_made"], box_team["two_points_attempted"])
if {"three_points_made","three_points_attempted"}.issubset(box_team.columns):
    box_team["fg3_pct"] = safe_div(box_team["three_points_made"], box_team["three_points_attempted"])
if {"free_throws_made","free_throws_attempted"}.issubset(box_team.columns):
    box_team["ft_pct"]  = safe_div(box_team["free_throws_made"], box_team["free_throws_attempted"])

# total rebounds if split is present
if {"offensive_rebounds","defensive_rebounds"}.issubset(box_team.columns) and "total_rebounds" not in box_team.columns:
    box_team["total_rebounds"] = box_team["offensive_rebounds"].fillna(0) + box_team["defensive_rebounds"].fillna(0)

# assist/turnover ratio (protect against 0)
if {"assists","turnovers"}.issubset(box_team.columns):
    box_team["ast_tov"] = box_team["assists"] / box_team["turnovers"].replace(0, np.nan)


### **6) Split into home vs away rows and pivot wide**

In [17]:
# Normalize ids to the same dtype
game_master[[TEAM_A, TEAM_B]] = game_master[[TEAM_A, TEAM_B]].astype(str)
box_team[BOX_TEAM] = box_team[BOX_TEAM].astype(str)

# HOME features
home_feats = game_master[[GAME_ID, TEAM_A, "season_code", "home_win"]].merge(
    box_team, left_on=[GAME_ID, TEAM_A, "season_code"], right_on=[BOX_GAME, BOX_TEAM, "season_code"], how="left"
)

# AWAY features
away_feats = game_master[[GAME_ID, TEAM_B, "season_code"]].merge(
    box_team, left_on=[GAME_ID, TEAM_B, "season_code"], right_on=[BOX_GAME, BOX_TEAM, "season_code"], how="left"
)

# Keep only numeric columns for features and prefix
home_num = home_feats.select_dtypes(include=[np.number]).add_prefix("home_")
away_num = away_feats.select_dtypes(include=[np.number]).add_prefix("away_")

# --- Safer version: works even if team names/abbr columns are missing ---
base_cols = [GAME_ID, TEAM_A, TEAM_B, "season_code", "home_win"]
optional_cols = ["home_team_name", "away_team_name", "home_abbr", "away_abbr"]
present_optionals = [c for c in optional_cols if c in game_master.columns]

games = game_master[base_cols + present_optionals].copy()
games = games.join(home_num.reset_index(drop=True)).join(away_num.reset_index(drop=True))

print("✅ Combined game-level dataset built.")
print("Shape:", games.shape)
print("Columns (sample):", games.columns[:15].tolist())

✅ Combined game-level dataset built.
Shape: (4667, 50)
Columns (sample): ['game_id', 'team_id_a', 'team_id_b', 'season_code', 'home_win', 'home_home_win', 'home_points', 'home_assists', 'home_steals', 'home_turnovers', 'home_blocks_favour', 'home_blocks_against', 'home_fouls_committed', 'home_fouls_received', 'home_offensive_rebounds']


### **7) Minimal missing-value handling (for now)**

We keep this conservative; more sophisticated imputation can come later.

In [18]:
# Fill NaNs in engineered rates with 0 (common, safe baseline), leave labels intact
feature_cols = games.select_dtypes(include=[np.number]).columns.tolist()
feature_cols.remove("home_win")
games[feature_cols] = games[feature_cols].fillna(0)

print("Games shape:", games.shape)
games.head(3)


Games shape: (4667, 50)


,game_id,team_id_a,team_id_b,season_code,home_win,home_home_win,home_points,home_assists,home_steals,home_turnovers,...,away_three_points_made,away_three_points_attempted,away_free_throws_made,away_free_throws_attempted,away_total_rebounds,away_valuation,away_fg2_pct,away_fg3_pct,away_ft_pct,away_ast_tov
0,E2007_001,OLY,BAS,E2007,1,1,190,32,20,22,...,24,62,24,30,81,173,0.477273,0.387097,0.800000,1.307692
1,E2007_002,VIR,ZAL,E2007,1,1,162,16,28,32,...,12,56,26,42,70,166,0.687500,0.214286,0.619048,0.666667
2,E2007_003,SOP,CSK,E2007,0,0,138,22,10,24,...,22,42,18,26,58,188,0.575000,0.523810,0.692308,2.125000


### **8) Save the training table**

In [19]:
OUT = Path("clean_data") / "games_features.csv"
games.to_csv(OUT, index=False)
print("✅ Saved:", OUT)


✅ Saved: clean_data\games_features.csv


In [20]:
# Sanity checks
assert "home_win" in games.columns, "Missing label 'home_win'."
print("Nulls per column (top 10):")
print(games.isna().sum().sort_values(ascending=False).head(10))

# (optional) fill NaNs in numeric features with 0 for baseline modeling
feature_cols = [c for c in games.columns if c not in [GAME_ID, TEAM_A, TEAM_B, "season_code", "home_win"]]
games[feature_cols] = games[feature_cols].fillna(0)


Nulls per column (top 10):
game_id                    0
away_defensive_rebounds    0
home_ast_tov               0
away_points                0
away_assists               0
away_steals                0
away_turnovers             0
away_blocks_favour         0
away_blocks_against        0
away_fouls_committed       0
dtype: int64



---

### 🧾 **Reflection — Stage 2: Data Integration & Feature Building**

In this stage, I transformed the cleaned Euroleague datasets into a unified, game-level analytics table ready for machine learning.

**Key accomplishments**

* **Integrated multi-source data:** Joined `header`, `boxscore`, and `teams` tables through validated keys (`game_id`, `team_id`, `season_code`), creating one consistent structure per game.
* **Aggregated player statistics:** Rolled player-level box scores up to team-level totals for each match, resolving duplicated and missing entries.
* **Engineered new metrics:** Calculated core efficiency ratios such as field-goal %, three-point %, free-throw %, rebound totals, and assist-to-turnover ratio.
* **Built home/away feature sets:** Pivoted team statistics into `home_*` and `away_*` columns and generated the binary label `home_win`.
* **Ensured reproducibility:** Filled minor numeric gaps, standardized data types, and exported a single modeling-ready file — `games_features.csv`.

> **Summary:**
> Stage 2 established the analytical backbone of the project. Raw multi-table data were reshaped into a structured, feature-rich dataset that captures each game’s statistical profile and outcome — the essential bridge between data engineering and predictive modeling.

---

